In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/MS_DS/NLP/Final project/nlp-text-summarisation


/content/drive/MyDrive/MS_DS/NLP/Final project/nlp-text-summarisation


In [3]:
pwd

'/content/drive/MyDrive/MS_DS/NLP/Final project/nlp-text-summarisation'

In [4]:
!source project-init.sh


Error: Command '['/content/drive/MyDrive/MS_DS/NLP/Final project/nlp-text-summarisation/env/bin/python3', '-Im', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
project-init.sh: line 7: ./env/bin/activate: No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow-macos (from versions: none)
ERROR: No matching distribution found for tensorflow-macos
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

repo = "google/pegasus-xsum"
tokenizer = AutoTokenizer.from_pretrained(repo)
model = AutoModelForSeq2SeqLM.from_pretrained(repo)

if torch.cuda.is_available():
  model.to('cuda')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)


In [7]:
def summarize(text, num_beams=4, no_repeat_ngram_size=2, min_length=30, max_length=130, early_stopping=True, skip_special_tokens=True, do_sample=False):
    preprocess_text = text.strip().replace("\n","")
    pegasus_prepared_Text = "summarize: "+preprocess_text
    #print ("original text preprocessed: \n", preprocess_text)

    tokenized_text = tokenizer.encode(pegasus_prepared_Text, return_tensors="pt", max_length=512).to(device)


    # summmarize 
    summary_ids = model.generate(tokenized_text,
                                   # num_beams=4,
                                   # no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

    output = tokenizer.decode(summary_ids[0], skip_special_tokens=skip_special_tokens)
    return output

In [20]:
import json
from generators import get_cnn_dm_both_generator

output = []

test_data_path = './dataset/chunked/test_000.bin'
for article, abstract in get_cnn_dm_both_generator(test_data_path):
    article_len = len(article)
    if article_len > 5000:
        #print(f'Skipping text - len={article_len}!')
        # Tokenizer cannot handle inputs longer than that
        continue

    #print(f'Summarizing text - len={article_len}')
    bart_abstract = summarize(article)
    output.append({
        'article': article,
        'abstract': abstract,
        'pegasus_abstract': bart_abstract
    })

with open('pegasus_output_000.json', 'w') as fout:
    json.dump(output, fout, indent=2)

example_generator completed reading all datafiles. No more data.


Test Results

In [21]:
# test output
with open('pegasus_output_000.json', 'r') as openfile:
 
    # Reading from json file
    json_object = json.load(openfile)

print("######### abstract #########")
print(json_object[30]['abstract'])
print("######### pegasus_abstract #########")
print(json_object[30]['pegasus_abstract'])



######### abstract #########
<s> high temperatures are recorded on the northern tip of the antarctica peninsula . </s> <s> the world meteorological organization will make the final determination . </s>
######### pegasus_abstract #########
The highest temperature ever recorded on antarctica is being investigated by the World Meteorological Organization to see if it's linked to climate change.


Eval results with rouge

In [25]:
import pandas as pd

df = pd.read_json('pegasus_output_000.json')


In [26]:
df

,article,abstract,pegasus_abstract
0,-lrb- cnn -rrb- the palestinian authority offi...,<s> membership gives the icc jurisdiction over...,palestinians have formally joined the Internat...
1,"-lrb- cnn -rrb- on may 28 , 2014 , some 7,000 ...",<s> amnesty international releases its annual ...,In our series of letters from African journali...
2,"-lrb- cnn -rrb- seventy years ago , anne frank...",<s> museum : anne frank died earlier than prev...,"The story of anne and margot frank, two Jewish..."
3,-lrb- cnn -rrb- a duke student has admitted to...,<s> student is no longer on duke university ca...,Hundreds of students and faculty members march...
4,-lrb- cnn -rrb- never mind cats having nine li...,"<s> theia , a bully breed mix , was apparently...",An online campaign has been launched to raise ...
...,...,...,...
707,-lrb- cnn -rrb- `` it really depends what you ...,<s> cnn team finds a man at `` unofficial '' d...,In our series of letters from African journali...
708,-lrb- cnn -rrb- every week in the heart of nim...,"<s> many girls in nima , one of accra 's poore...",In our series of letters from African journali...
709,-lrb- cnn -rrb- a florida judge denied a motio...,<s> judge also throws out bombshell sex claims...,The Duke of york has been cleared of allegatio...
710,-lrb- cnn -rrb- at least 54 people have died a...,<s> fishing vessels are searching for 15 peopl...,At least 15 people have died and 15 others are...


In [13]:
!pip install rouge


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from rouge import Rouge

rouge = Rouge()


In [27]:
pred_str = df['pegasus_abstract']
label_str = df['abstract']

rouge_output = rouge.get_scores(pred_str, label_str)

print(rouge_output)

[{'rouge-1': {'r': 0.30303030303030304, 'p': 0.43478260869565216, 'f': 0.357142852302296}, 'rouge-2': {'r': 0.10526315789473684, 'p': 0.17391304347826086, 'f': 0.1311475362859448}, 'rouge-l': {'r': 0.2727272727272727, 'p': 0.391304347826087, 'f': 0.3214285665880103}}, {'rouge-1': {'r': 0.1388888888888889, 'p': 0.22727272727272727, 'f': 0.17241378839476829}, 'rouge-2': {'r': 0.075, 'p': 0.125, 'f': 0.09374999531250022}, 'rouge-l': {'r': 0.1111111111111111, 'p': 0.18181818181818182, 'f': 0.13793102977407862}}, {'rouge-1': {'r': 0.25925925925925924, 'p': 0.2413793103448276, 'f': 0.24999999500637762}, 'rouge-2': {'r': 0.029411764705882353, 'p': 0.034482758620689655, 'f': 0.03174602677752661}, 'rouge-l': {'r': 0.18518518518518517, 'p': 0.1724137931034483, 'f': 0.17857142357780628}}, {'rouge-1': {'r': 0.2972972972972973, 'p': 0.4583333333333333, 'f': 0.36065573293200753}, 'rouge-2': {'r': 0.02127659574468085, 'p': 0.03571428571428571, 'f': 0.02666666198755637}, 'rouge-l': {'r': 0.21621621621